## Neuromorphic HAR

### By loading optimized (hyper)parameters and weights, the confusion matrices can be here obtained, together with memory and energy evaluations.

#### <b>IMPORTANT NOTES:</b>
<b>1)</b> for each network,  variables <b>optim_nni_experiment</b> and <b>optim_nni_trial</b> must be set accordingly to the IDs of the NNI optimization experiment and trial whose results are to be used;<br>
<b>2)</b> in the case of the sCNN, these variables are used to keep the optimized structure parameters as for the <i>NON-SPIKING</i> counterpart. The NNI optimization experiment and trial IDs for the spiking CNN must be set through the variables <b>snn_nni_experiment</b> and <b>snn_nni_trial</b> respectively;<br>
<b>3)</b> NNI experiment results (of each trial) can be found in:<br>
&emsp;&emsp;{os.path.expanduser('~')}<br>
&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;| \_ \_ nni-experiments<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;| \_ \_  {experiment ID}<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;| \_ \_  trials<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;|<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;| \_ \_  {trial ID}<br>

<b>Hyperlinks to each network:</b><br>
<b>[LSTM](#Section_1)</b><br>
<b>[CNN](#Section_2)</b><br>
<b>[sCNN](#Section_3)</b><br>
<b>[LMU](#Section_4)</b><br>
<b>[sLMU](#Section_5)</b><br>
<b>[LMU (ff)](#Section_6)</b><br>
<b>[sLMU (ff)](#Section_7)</b><br>
[FLOPs calculation and energy estimation for LMU and LMU (ff)](#Section_8)<br>

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = 'true'

In [3]:
import numpy as np
import pandas as pd
import statistics
import csv
import itertools
import json
import random as rn
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.request import urlretrieve

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

from scipy.signal import butter, freqz

import nengo
import nengo_dl
from nengo.utils.filter_design import cont2discrete
import tensorflow as tf
import keras
from keras.callbacks import Callback
from keras.models import Sequential
from keras.models import Model
from keras.layers import *
from keras.utils import to_categorical
from keras.regularizers import l2,l1
from keras.optimizers import Adam

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph

2023-09-16 11:49:34.416921: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-16 11:49:35.968119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/root/.mujoco/mujoco210/bin:/usr/lib/nvidia-520
2023-09-16 11:49:35.978807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/root/.mujoco/mujoco210/bin:/usr/lib/nvidia-520
2023-09-16 11:49:35.978830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please m

In [4]:
class LMUCell(nengo.Network):
    def __init__(self, units, order, theta, input_d, tau, **kwargs):
        super().__init__(**kwargs)

        Q = np.arange(order, dtype=np.float64)
        R = (2 * Q + 1)[:, None] / theta
        j, i = np.meshgrid(Q, Q)

        A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R 
        B = (-1.0) ** Q[:, None] * R 
        C = np.ones((1, order))
        D = np.zeros((1,))

        A, B, _, _, _ = cont2discrete((A, B, C, D), dt=tau, method="zoh") # original: dt=1.0

        A_H = 1/(1-np.exp(-1/tau)) * (A - np.exp(-1/tau)*np.identity(order))
        B_H = 1/(1-np.exp(-1/tau)) * B


        with self:
            nengo_dl.configure_settings(trainable=None)

            # create objects corresponding to the x/u/m/h
            self.x = nengo.Node(size_in=input_d)
            self.u = nengo.Node(size_in=1)
            self.m = nengo.Node(size_in=order)
            self.h = nengo_dl.TensorNode(tf.nn.tanh, shape_in=(units,), pass_time=False)

            # compute u_t:
            # e_x
            nengo.Connection(
                self.x, self.u, transform=np.ones((1, input_d)), synapse=None
            )
            
            # e_h
            nengo.Connection(
                self.h, self.u, transform=np.ones((1, units)), synapse=0
            )
            
            # e_m
            nengo.Connection(
                self.m, self.u, transform=np.ones((1, order)), synapse=0
            )

            # compute m_t:
            conn_A = nengo.Connection(self.m, self.m, transform=A_H, synapse=0)
            self.config[conn_A].trainable = True
            conn_B = nengo.Connection(self.u, self.m, transform=B_H, synapse=None)
            self.config[conn_B].trainable = True

            # compute h_t:
            nengo.Connection(
                self.x, self.h, transform=nengo_dl.dists.Glorot(), synapse=None
            )
            nengo.Connection(
                self.h, self.h, transform=nengo_dl.dists.Glorot(), synapse=0
            )
            nengo.Connection(
                self.m, self.h, transform=nengo_dl.dists.Glorot(), synapse=None,
            )

In [5]:
def load_wisdm2_data(filename):
    filepath = os.path.join('./data/',filename+'.npz')
    a = np.load(filepath)
    return (a['arr_0'], a['arr_1'], a['arr_2'], a['arr_3'], a['arr_4'], a['arr_5'])

In [6]:
class DeviceData:
    def __init__(self, sample, fs, channels):
        self.data = []
        sample = sample.T
        for data_axis in range(sample.shape[0]):
            self.data.append(sample[data_axis, :])

        self.fs = fs
        self.freq_range = (0.5, np.floor(self.fs / 2))

        freq_min, freq_max = self.freq_range
        octave = (channels - 0.5) * np.log10(2) / np.log10(freq_max / freq_min)
        self.freq_centr = np.array([freq_min * (2 ** (ch / octave)) for ch in range(channels)])
        self.freq_poli = np.array(
            [(freq * (2 ** (-1 / (2 * octave))), (freq * (2 ** (1 / (2 * octave))))) for freq in self.freq_centr])
        self.freq_poli[-1, 1] = fs / 2 * 0.99999

    def decomposition(self, filterbank):
        self.components = []
        for data_axis in self.data:
            tmp = []
            for num, den in filterbank:
                from scipy.signal import lfilter
                tmp.append(lfilter(num, den, data_axis))
            self.components.append(tmp)


def frequency_decomposition(array, channels=5, fs=20, order=2):

    array_dec = []

    for ii in range(len(array)):
    
        sample = DeviceData(array[ii], fs, channels)
    
        butter_filterbank = []
        for fl, fh in sample.freq_poli:
            num, den = butter(N=order, Wn=(fl, fh), btype='band', fs=sample.fs)
            butter_filterbank.append([num, den])
    
        sample.decomposition(butter_filterbank)
    
        features = []
        for data_axis in sample.components:
            for component in data_axis:
                features.append(np.array(component))
        features = np.vstack(features)
        features = features.T
    
        array_dec.append(features)

    return np.array(array_dec)

In [7]:
# Plot loss and accuracy at the end of training
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend(['train_'+string, 'val_'+string])
    plt.show()

In [8]:
def ConfusionMatrix_labels(subset):
    
    act_map = {
        'A': 'walking',
        'B': 'jogging',
        'C': 'stairs',
        'D': 'sitting',
        'E': 'standing',
        'M': 'kicking',
        'P': 'dribbling',
        'O': 'catch',
        'F': 'typing',
        'Q': 'writing',
        'R': 'clapping',
        'G': 'teeth',
        'S': 'folding',
        'J': 'pasta',
        'H': 'soup',
        'L': 'sandwich',
        'I': 'chips',
        'K': 'drinking',
    }
    
    if subset == 1:
        labels = list(act_map.values())[:6]
    if subset == 2:
        labels = list(act_map.values())[6:13]
    if subset == 3:
        labels = list(act_map.values())[13:]
    
    return labels

In [9]:
def memory_footprint(model, nengo=True):
    
    mem_fp = 0
    total = 0
    missed = 0
    
    if nengo:
        
        model_weights = model.keras_model.weights
    
    else:
        
        model_weights = model.weights
        
    for s in model_weights:
        if ('32' in str(s.dtype)) or (str(s.dtype)=='int'):
            mem = 4*np.prod(s.shape)/1e6 # MB
            total += np.prod(s.shape)
            mem_fp += mem
        elif '64' in str(s.dtype):
            mem = 8*np.prod(s.shape)/1e6 # MB
            total += np.prod(s.shape)
            mem_fp += mem
        else:
            missed += np.prod(s.shape)
    
    return mem_fp, total, missed

In [10]:
def get_flops(model):
    
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function([tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func)
    
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        
        return flops.total_float_ops

In [11]:
def get_sops_LMUens(net, freqdec=False):
    
    with net:
        lmu_inner.add_neuron_output()
        p_spikes = nengo.Probe(lmu_inner.neuron_output, label="p_spikes")
        net.config[p_spikes].keep_history = True
    
    with nengo_dl.Simulator(net) as sim:
        
        sim.load_params("./output/tmp_{}_{}_{}/best_test_{}".format(net_type,optim_nni_experiment,datafile[5:],optim_nni_experiment))
    
        sops = []
        accs = []
        
        dt = 0.001 # the default value in Nengo
    
        for ii in range(int(0.1*len(x_test))):
            
            simulation_steps = int(len(x_test[ii]))
    
            sim.run_steps(simulation_steps, data={inp: x_test[ii][np.newaxis,:,:]})
    
        spikes = sim.data[p_spikes]/amplitude*dt
        spikes_per_neuron = np.sum(spikes > 0, axis=0)
        sops = np.sum(spikes_per_neuron)/int(0.1*len(x_test))
    
        energy = sops*5.07e-10 # Event-Driven Signal Processing with Neuromorphic Computing Systems, https://ieeexplore.ieee.org/document/9053043/
    
        num_nn = 0
        for ee in net.all_ensembles:
            for nn in ee.neurons:
                num_nn +=1
        
        print("\n")
        print("Total number of neurons:",int(num_nn))
        print("SOPs:",int(np.round(np.mean(sops),0)))
        print("Energy evaluation on Loihi: "+str(np.round(energy*1e6,2))+" μJ")

In [12]:
x_bonito=np.random.rand(1,4,2000)*255
def get_sops_spikingCNN(net):
    
    with net:
        
        dense_p = nengo.Probe(net.layers[model.layers[5].get_output_at(-1)])
    
    with nengo_dl.Simulator(net) as sim:
        
        #sim.load_params("./output/tmp_s{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
    
        sops = []
        preds = []
        accs = []
        
        dt = 0.001 # the default value in Nengo
    
        for ii in range(int(0.1*len(x_bonito))): #tiled_x_test
            
            simulation_steps = int(len(x_test[ii])) #tiled_x_test
        
            sim.run_steps(simulation_steps, data={net.all_nodes[0]: x_bonito[ii][np.newaxis,:,:]}) #tiled_x_test
    
        spikes_conv0 = sim.data[conv0_p]/(1/100)*dt #snn_parameters['nni_keras2snn_network/scale_firing_rates/randint']
        spikes_conv1 = sim.data[conv1_p]/(1/100)*dt
        spikes_dense = sim.data[dense_p]/(1/100)*dt
        spikes_per_neuron_conv0 = np.sum(spikes_conv0 > 0, axis=0)
        spikes_per_neuron_conv1 = np.sum(spikes_conv1 > 0, axis=0)
        spikes_per_neuron_dense = np.sum(spikes_dense > 0, axis=0)
        sops = np.sum([np.sum(spikes_per_neuron_conv0), np.sum(spikes_per_neuron_conv1), np.sum(spikes_per_neuron_dense)]) / int(0.1*len(x_bonito)) #tiled_x_test
    
        energy = sops*5.07e-10 # Event-Driven Signal Processing with Neuromorphic Computing Systems, https://ieeexplore.ieee.org/document/9053043/
        
        num_nn = 0
        for ee in net.all_ensembles:
            for nn in ee.neurons:
                num_nn +=1
        
        print("\n")
        print("Total number of neurons:",int(num_nn))
        print("SOPs:",int(np.round(np.mean(sops),0)))
        print("Energy evaluation on Loihi: "+str(np.round(energy*1e6,2))+" μJ")

<a id='Section_1'></a>
### LSTM

In [13]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [14]:
net_type = 'lstm'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [15]:
model_name = "LSTM_{}_subset{}_{}".format(device,subset,window_size)

In [16]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open("./fakeparams.cfg", 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['nni_network/batch_size/randint']
###########################################

In [17]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = y_train_oh
y_val = y_val_oh
y_test = y_test_oh

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)
x_test.shape
len(x_test)

timesteps: 40
input_dim: 6
n_classes: 7


7241

In [18]:
# Initiliazing the sequential model
model = Sequential()
# First LSTM layer
model.add(LSTM(network_parameters['nni_network/LSTM_units_1/randint'],
               return_sequences=True,
               input_shape=(timesteps, input_dim))
        )
# Adding a dropout layer
model.add(Dropout(network_parameters['nni_network/LSTM_Dropout_1/quniform']))
# Second LSTM layer
model.add(LSTM(network_parameters['nni_network/LSTM_units_2/randint'],
               recurrent_regularizer=l2(network_parameters['nni_network/LSTM_l2_2/quniform']), 
               input_shape=(timesteps, input_dim))
         ) 
# Adding a dropout layer
model.add(Dropout(network_parameters['nni_network/LSTM_Dropout_2/quniform']))
# Adding a dense output layer
model.add(Dense(n_classes, activation='softmax')) 
model.summary()

# Compiling the model
optim = Adam(lr=network_parameters['nni_network/lr/quniform'])
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

2023-09-16 11:50:52.264515: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 128)           69120     
                                                                 
 dropout (Dropout)           (None, 40, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 7)                 903       
                                                                 
Total params: 201,607
Trainable params: 201,607
Non-trainable params: 0
_________________________________________________________________


/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
model=Sequential()

#inp = tf.keras.Input(shape=( 1, 2000)) #4, 2000

conv0 = tf.keras.layers.Conv1D(
    filters=4,
    kernel_size=5,
    strides=1,
    padding="same",
    activation=tf.nn.silu,
)
model.add(conv0)

conv1 = tf.keras.layers.Conv1D(
    filters=16,
    kernel_size=5,
    strides=1,
    padding="same",
    activation=tf.nn.silu,
)
model.add(conv1)

conv2 = tf.keras.layers.Conv1D(
    filters=384,
    kernel_size=19,
    strides=5,
    padding="same",
    activation=tf.nn.silu,
)
model.add(conv2)
"""
for i in range(5):
    model.add(LSTM(384,input_shape=(4,400,384), return_sequences=True))
    pass
"""
#k_model2 = tf.keras.Model(inputs=inp, outputs=conv2)
optim = Adam(lr=network_parameters['nni_network/lr/quniform'])
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])
model.build(( 1,2000,1))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (1, 2000, 4)              24        
                                                                 
 conv1d_4 (Conv1D)           (1, 2000, 16)             336       
                                                                 
 conv1d_5 (Conv1D)           (1, 400, 384)             117120    
                                                                 
Total params: 117,480
Trainable params: 117,480
Non-trainable params: 0
_________________________________________________________________


In [22]:
mem_fp, total, missed = memory_footprint(model, nengo=False)

print('Memory footprint (MB):',np.round(mem_fp,4))
print('Total:',total)
print('Missed:',missed)

Memory footprint (MB): 0.4699
Total: 117480
Missed: 0


In [24]:
print("FLOPs: {}".format(get_flops(model)))

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated

In [25]:
energy = get_flops(model)*7.53e-10 # Event-Driven Signal Processing with Neuromorphic Computing Systems, https://ieeexplore.ieee.org/document/9053043/

print("Energy evaluation on Movidius: "+str(np.round(energy*1e6,2))+" μJ")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Energy evaluation on Movidius: 71783.19 μJ

Doc:
op: The nodes are operation kernel type, such as MatMul, Conv2D. Graph nodes belonging to the same type are aggregated together.
flops: Number of float operations. 

In [72]:
model.load_weights("./output/tmp_{}_{}_{}/best_test/best_test_{}".format(net_type,optim_nni_experiment,datafile[5:],optim_nni_experiment))

_, acc = model.evaluate(x_test, y_test, batch_size=minibatch_train)
print("Test accuracy: "+str(np.round(acc*100,2))+"%")

pred = model.predict(x_test, batch_size=minibatch_train)

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./output/tmp_lstm__watch_subset2_40/best_test/best_test_

In [73]:
save = False

cm = confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1), normalize='true')
labels = ConfusionMatrix_labels(subset)
cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
plt.figure(figsize=(7,5.25))
sn.heatmap(cm_df,
           annot=True,
           fmt='.2g',
           cbar=False,
           square=False,
           cmap="YlGnBu")
plt.xlabel('\nPredicted')
plt.ylabel('True\n')
plt.yticks(rotation=0)
plt.title('LSTM\n', fontweight='bold', fontsize=16)
plt.tight_layout()
if save:
    plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
plt.show()

NameError: name 'pred' is not defined

<a id='Section_2'></a>
### CNN

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
net_type = 'cnn'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
model_name = "CNN_{}_subset{}_{}".format(device,subset,window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['nni_network/batch_size/randint']
###########################################

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = y_train_oh
y_val = y_val_oh
y_test = y_test_oh

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

In [ ]:
# Initiliazing the sequential model
model = Sequential()
# First convolutional layer
model.add(Conv1D(filters=network_parameters['nni_network/Conv1D_filters_1/randint'], 
                 kernel_size=network_parameters['nni_network/Conv1D_kernel_size_1/randint'], 
                 activation='relu',
                 kernel_initializer='he_uniform',
                 input_shape=(timesteps,input_dim))
         )
# Second convolutional layer
model.add(Conv1D(filters=network_parameters['nni_network/Conv1D_filters_2/randint'], 
                 kernel_size=network_parameters['nni_network/Conv1D_kernel_size_2/randint'], 
                 activation='relu',
                 kernel_initializer='he_uniform')
         )
# Adding a pooling layer
model.add(MaxPooling1D(pool_size=2))
# Adding a flattening layer
model.add(Flatten())
# Adding a dense layer
model.add(Dense(network_parameters['nni_network/CNN_Dense_1/randint'], 
                activation='relu')
         )
# Adding a dense output layer
model.add(Dense(n_classes, activation='softmax'))
model.summary()

# Compiling the model
optim = Adam(lr=network_parameters['nni_network/lr/quniform'])
model.compile(loss='categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])

In [ ]:
mem_fp, total, missed = memory_footprint(model, nengo=False)

print('Memory footprint (MB):',np.round(mem_fp,4))
print('Total:',total)
print('Missed:',missed)

In [ ]:
print("FLOPs: {}".format(get_flops(model)))

In [ ]:
energy = get_flops(model)*7.53e-10 # Event-Driven Signal Processing with Neuromorphic Computing Systems, https://ieeexplore.ieee.org/document/9053043/

print("Energy evaluation on Movidius: "+str(np.round(energy*1e6,2))+" μJ")

In [ ]:
model.load_weights("./output/tmp_{}_{}_{}/best_test/best_test_{}".format(net_type,optim_nni_experiment,datafile[5:],optim_nni_experiment))

_, acc = model.evaluate(x_test, y_test, batch_size=minibatch_train)
print("Test accuracy: "+str(np.round(acc*100,2))+"%")

pred = model.predict(x_test, batch_size=minibatch_train)

In [ ]:
save = False

cm = confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1), normalize='true')
labels = ConfusionMatrix_labels(subset)
cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
plt.figure(figsize=(7,5.25))
sn.heatmap(cm_df,
           annot=True,
           fmt='.2g',
           cbar=False,
           square=False,
           cmap="YlGnBu")
plt.xlabel('\nPredicted')
plt.ylabel('True\n')
plt.yticks(rotation=0)
plt.title('CNN\n', fontweight='bold', fontsize=16)
plt.tight_layout()
if save:
    plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
plt.show()

<a id='Section_3'></a>
### Spiking CNN

In [74]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [75]:
network_type = 'cnn'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [76]:
model_name = "sCNN_{}_subset{}_{}".format(device,subset,window_size)

In [98]:
##### GET NETWORK STRUCTURE PARAMETERS from NNI-optimized non-spiking CNN #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['nni_network/batch_size/randint']
##############################################################################


##### GET SNN PARAMETERS #####
snn_nni_experiment = ''
snn_nni_trial = '' 
snn_filename = 'parameter.cfg'
snn_nni_ref = 'nni-experiments/'+snn_nni_experiment+'/trials/'+snn_nni_trial
snn_nni_dir = os.path.expanduser('~')
snn_filepath = os.path.join(snn_nni_dir,snn_nni_ref,snn_filename)

with open(snn_filepath, 'r') as f:
    snn_data = f.read()

snn_params = json.loads(snn_data)
snn_parameters = snn_params['parameters']
##############################

FileNotFoundError: [Errno 2] No such file or directory: '/root/nni-experiments//trials/parameter.cfg'

In [82]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

### flatten data and add time dimension:
x_train = x_train.reshape((x_train.shape[0], 1, -1))
y_train = y_train[:,None,None]
x_val = x_val.reshape((x_val.shape[0], 1, -1))
y_val = y_val[:,None,None]
x_test = x_test.reshape((x_test.shape[0], 1, -1))
y_test = y_test[:,None,None]

timesteps: 40
input_dim: 6
n_classes: 7


In [78]:
model_nonspiking = Sequential()    
model_nonspiking.add(Conv1D(filters=network_parameters['nni_network/Conv1D_filters_1/randint'], kernel_size=network_parameters['nni_network/Conv1D_kernel_size_1/randint'], activation=tf.nn.relu, kernel_initializer='he_uniform', input_shape=(timesteps,input_dim), name='Conv1D_1'))
model_nonspiking.add(Conv1D(filters=network_parameters['nni_network/Conv1D_filters_2/randint'], kernel_size=network_parameters['nni_network/Conv1D_kernel_size_2/randint'], activation=tf.nn.relu, kernel_initializer='he_uniform', name='Conv1D_2'))
model_nonspiking.add(MaxPooling1D(pool_size=2, name='MaxPooling1D'))
model_nonspiking.add(Flatten())
model_nonspiking.add(Dense(network_parameters['nni_network/CNN_Dense_1/randint'], activation=tf.nn.relu, name='Dense_1'))
model_nonspiking.add(Dense(n_classes, activation='softmax', name='Dense_2'))
    
### LOAD PRE-TRAINED WEIGHTS from NNI-optimized non-spiking CNN ###
model_nonspiking.load_weights("./output/tmp_cnn_{}_{}/best_test/best_test_{}".format(net_type,optim_nni_experiment,datafile[5:],optim_nni_experiment))

### sequential to functional model
input_layer = Input(batch_shape=model_nonspiking.layers[0].input_shape, name='Input')
prev_layer = input_layer
for num,el in enumerate(model_nonspiking.layers):
    prev_layer = el(prev_layer)

model = Model([input_layer], [prev_layer])

model.summary()

KeyError: 'nni_network/Conv1D_filters_1/randint'

In [83]:
keras_layers = list(model.layers[ii].name for ii in range(len(model.layers)))

In [84]:
trained_converter = nengo_dl.Converter(model,
                                       max_to_avg_pool=True,
                                       swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                                       scale_firing_rates=snn_parameters['nni_keras2snn_network/scale_firing_rates/randint'],
                                       synapse=snn_parameters['nni_keras2snn_network/synapse/quniform'],
                                       )

print('\n##### neuron type now is:')
for ii in range(len(trained_converter.net.ensembles)):
    print('In ensemble',ii,':',trained_converter.net.ensembles[ii].neuron_type)
print('#########################\n')

NameError: name 'snn_parameters' is not defined

In [128]:
input_layer = Input(batch_shape=model.layers[0].input_shape, name='Input')
prev_layer = input_layer
for num,el in enumerate(model.layers):
    prev_layer = el(prev_layer)

model = Model([input_layer], [prev_layer])

model.summary()
#conversione modello keras bonitosnn
trained_converter = nengo_dl.Converter(model,
                                       max_to_avg_pool=True,
                                       swap_activations={tf.nn.silu: nengo.SpikingRectifiedLinear()},
                                       scale_firing_rates=100,
                                       synapse=1e-2,
                                       )

print('\n##### neuron type now is:')
for ii in range(len(trained_converter.net.ensembles)):
    print('In ensemble',ii,':',trained_converter.net.ensembles[ii].neuron_type)
print('#########################\n')

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(1, 2000, 1)]            0         
                                                                 
 conv1d_86 (Conv1D)          (1, 2000, 4)              24        
                                                                 
 conv1d_87 (Conv1D)          (1, 2000, 16)             336       
                                                                 
 conv1d_88 (Conv1D)          (1, 400, 384)             117120    
                                                                 
 lstm_74 (LSTM)              (1, 400, 384)             1181184   
                                                                 
 lstm_75 (LSTM)              (1, 400, 384)             1181184   
                                                                 
 lstm_76 (LSTM)              (1, 400, 384)             1181

/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/nengo_dl/converter.py:324: UserWarning: Layer type LSTM does not have a registered converter. Falling back to TensorNode.
  warnings.warn(



##### neuron type now is:
In ensemble 0 : SpikingRectifiedLinear(amplitude=0.01)
In ensemble 1 : SpikingRectifiedLinear(amplitude=0.01)
In ensemble 2 : SpikingRectifiedLinear(amplitude=0.01)
#########################



In [149]:
print(model.output)
for k in trained_converter.outputs.keys():
    print(k)
model.layers[1].get_output_at(-1)

KerasTensor(type_spec=TensorSpec(shape=(1, 400, 384), dtype=tf.float32, name=None), name='lstm_78/PartitionedCall:1', description="created by layer 'lstm_78'")
(<Reference wrapping <KerasTensor: shape=(1, 400, 384) dtype=float32 (created by layer 'lstm_78')>>,)


<KerasTensor: shape=(1, 2000, 4) dtype=float32 (created by layer 'conv1d_86')>

In [158]:
with trained_converter.net:
    output_p = trained_converter.outputs[model.output]
    conv0_p = nengo.Probe(trained_converter.layers[model.layers[1].get_output_at(-1)])
    conv1_p = nengo.Probe(trained_converter.layers[model.layers[2].get_output_at(-1)])
    conv2_p = nengo.Probe(trained_converter.layers[model.layers[3].get_output_at(-1)])

In [131]:
n_steps = snn_parameters['nni_keras2snn_network/n_steps/randint']

tiled_x_test = np.tile(x_test, (1, n_steps, 1))

NameError: name 'snn_parameters' is not defined

In [159]:
with nengo_dl.Simulator(trained_converter.net, minibatch_size=1) as sim:
    snn_model_summary = sim.keras_model
    snn_params = sum(np.prod(s.shape) for s in snn_model_summary.weights)
    snn_trainable_params = sum(np.prod(w.shape) for w in snn_model_summary.trainable_weights)
    print('\n=================================================================')
    print('Total params:','{:,d}'.format(snn_params))
    print('Trainable params:','{:,d}'.format(snn_trainable_params))
    print('Non-trainable params:','{:,d}'.format(snn_params-snn_trainable_params))
    print('_________________________________________________________________\n')
    
    mem_fp, total, missed = memory_footprint(sim)

    print('Memory footprint (MB):',np.round(mem_fp,4))
    print('Total:',total)
    print('Missed:',missed)
    print("\n")
    
    sim.compile(
                optimizer=tf.optimizers.Adam(0.0003),
                loss={
                      output_p: tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                      conv0_p: tf.losses.mse,
                      conv1_p: tf.losses.mse,
                     },
                loss_weights={
                              output_p: 1, 
                              conv0_p: 1e-3,#snn_parameters['nni_keras2snn_network/reg_conv0/quniform'], 
                              conv1_p: 1e-3,#snn_parameters['nni_keras2snn_network/reg_conv1/quniform']
                              conv1_p: 1e-3,
                             },
                metrics=["accuracy"],
               )
    
    #sim.load_params("./output/tmp_s{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
        
    data = sim.predict({trained_converter.inputs[model.input]: x_bonito}) #tiled_x_test
    predictions = np.argmax(data[trained_converter.outputs[model.output]][:, -1], axis=-1)
    test_accuracy = (predictions[:] == y_test[:predictions.shape[0], 0, 0]).mean()
    print("Test accuracy: "+str(np.round(test_accuracy*100,2))+"%")
    
    save = True
    """
    cm = confusion_matrix(y_test[:np.min([len(y_test), len(predictions)]),-1,-1], predictions[:np.min([len(y_test), len(predictions)])], normalize='true')
    labels = ConfusionMatrix_labels(subset)
    cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
    plt.figure(figsize=(7,5.25))
    sn.heatmap(cm_df,
               annot=True,
               fmt='.2g',
               cbar=False,
               square=False,
               cmap="YlGnBu")
    plt.xlabel('\nPredicted')
    plt.ylabel('True\n')
    plt.yticks(rotation=0)
    plt.title('Spiking CNN\n', fontweight='bold', fontsize=16)
    plt.tight_layout()
    if save:
        plt.savefig('./pictures/'+model_name+'_'+snn_nni_experiment+'_'+snn_nni_trial+'.png')
    plt.show()
    """
sim.close()

|                     Building network (0%)                    | ETA:  --:--:--


|#                     Building network (2%)                     | ETA: 0:00:02
|#                     Building network (2%)                     | ETA: 0:00:04
|#                     Building network (2%)                     | ETA: 0:00:06
|#                     Building network (2%)                     | ETA: 0:00:09


/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/nengo_dl/simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(


|##                    Building network (4%)                     | ETA: 0:00:05
|##                    Building network (4%)                     | ETA: 0:00:06
|##                    Building network (4%)                     | ETA: 0:00:07
|##                    Building network (4%)                     | ETA: 0:00:08
|##                    Building network (4%)                     | ETA: 0:00:10
|##                    Building network (4%)                     | ETA: 0:00:11
|##                    Building network (4%)                     | ETA: 0:00:12
|##                    Building network (4%)                     | ETA: 0:00:13
|##                    Building network (4%)                     | ETA: 0:00:14
|##                    Building network (4%)                     | ETA: 0:00:15
|##                    Building network (4%)                     | ETA: 0:00:16
|##                    Building network (4%)                     | ETA: 0:00:17
|##                    Building network 

/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer Orthogonal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


|    #                    Constructing graph                          | 0:00:00
|     #                   Constructing graph                          | 0:00:00
|           Constructing graph: pre-build stage (0%)           | ETA:  --:--:--
|      #                  Constructing graph                          | 0:00:00
Constructing graph: pre-build stage finished in 0:00:00
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|##            Constructing graph: build stage (4%)              | ETA: 0:00:05
|########      Constructing graph: build stage (13%)             | ETA: 0:00:01
|###########   Constructing graph: build stage (18%)             | ETA: 0:00:01
|##############Constructing graph: build stage (22%)             | ETA: 0:00:01
|##############Constructing graph: build stage (27%)            

/root/miniconda3/envs/base_arch2/lib/python3.8/site-packages/nengo_dl/simulator.py:1765: UserWarning: Number of elements (1) in ['str'] does not match number of Probes (16); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  warnings.warn(


|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|             Constructing graph: build stage (0%)             | ETA:  --:--:--
|##            Constructing graph: build stage (4%)              | ETA: 0:00:05
|########      Constructing graph: build stage (13%)             | ETA: 0:00:01
|###########   Constructing graph: build stage (18%)             | ETA: 0:00:01
|##############Constructing graph: build stage (25%)             | ETA: 0:00:01
|##############Constructing graph: build stage (31%)             | ETA: 0:00:00
|##############Constructing graph: build stage (34%)             | ETA: 0:00:00
|##############Constructing graph: build stage (34%)             | ETA: 0:00:02
|##############Constructing graph: build stage (34%)             | ETA: 0:00:02
|##############Constructing graph: build stage (34%)             | ETA: 0:00:02
|##############Constructing graph: build stage (34%)             | ETA: 0:00:02
|##############Constructing graph: build

1/1 [==============================] - 9s 9s/step
Test accuracy: 0.0%


In [152]:
get_sops_spikingCNN(trained_converter.net)

|                     Building network (0%)                    | ETA:  --:--:--
|#                     Building network (2%)                     | ETA: 0:00:02
|#                     Building network (2%)                     | ETA: 0:00:04
|#                     Building network (2%)                     | ETA: 0:00:06
|#                     Building network (2%)                     | ETA: 0:00:08
|##                    Building network (4%)                     | ETA: 0:00:05
|##                    Building network (4%)                     | ETA: 0:00:06
|##                    Building network (4%)                     | ETA: 0:00:07
|##                    Building network (4%)                     | ETA: 0:00:08
|##                    Building network (4%)                     | ETA: 0:00:09
|##                    Building network (4%)                     | ETA: 0:00:10
|##                    Building network (4%)                     | ETA: 0:00:11
|##                    Building network 

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [157]:
sim.data[conv0_p]

[]

<a id='Section_4'></a>
### LMU

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
net_type = 'LMU'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
model_name = "LMU_{}_subset{}_{}".format(device,subset_window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['minibatch']
###########################################

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

y_train = y_train[:, None, None]
y_test = y_test[:, None, None]
y_val = y_val[:, None, None]

In [ ]:
with nengo.Network(seed=seed) as net:
    # remove some unnecessary features to speed up the training
    nengo_dl.configure_settings(
                                trainable=None,
                                stateful=False,
                                keep_history=False,
                               )

    # input node
    inp = nengo.Node(np.zeros(input_dim))

    # lmu cell
    lmu = LMUCell(
                  units=int(network_parameters['units']), 
                  order=int(network_parameters['order']), 
                  theta=network_parameters['theta'],
                  input_d=input_dim,
                  tau=network_parameters['tau'],
                )
    conn_in = nengo.Connection(inp, lmu.x, synapse=network_parameters['synapse_in'])
    net.config[conn_in].trainable = True

    # dense linear readout
    out = nengo.Node(size_in=n_classes)
    conn_out = nengo.Connection(lmu.h, out, transform=nengo_dl.dists.Glorot(), synapse=network_parameters['synapse_out'])
    net.config[conn_out].trainable = True

    # record output
    p = nengo.Probe(out)

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=minibatch_train) as sim:
    
    lmu_model_summary = sim.keras_model
    lmu_params = sum(np.prod(s.shape) for s in lmu_model_summary.weights)
    lmu_trainable_params = sum(np.prod(w.shape) for w in lmu_model_summary.trainable_weights)
    mem_fp, total, missed = memory_footprint(sim) 
    print('\n=================================================================')
    print('Total params:','{:,d}'.format(lmu_params))
    print('Trainable params:','{:,d}'.format(lmu_trainable_params))
    print('Non-trainable params:','{:,d}'.format(lmu_params-lmu_trainable_params))
    print('_________________________________________________________________\n')
    
    mem_fp, total, missed = memory_footprint(sim)

    print('Memory footprint (MB):',np.round(mem_fp,4))
    print('Total:',total)
    print('Missed:',missed)
    print("\n")
    
    sim.compile(
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam(network_parameters['lr']),
                metrics=["accuracy"],
               )
    
    sim.load_params("./output/tmp_{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
    
    test = sim.evaluate(x_test, y_test)["probe_accuracy"]
    print("Test accuracy: "+str(np.round(test*100,2))+"%")
    
    prediction = sim.predict(x_test)
        
    predictions = list(prediction.values())[0]
    pred = predictions.argmax(axis=-1)
    
    save = False
    
    cm = confusion_matrix(y_test[:np.min([len(y_test), len(pred)]),-1,-1], pred[:np.min([len(y_test), len(pred)]),-1], normalize='true')
    labels = ConfusionMatrix_labels(subset)
    cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
    plt.figure(figsize=(7,5.25))
    sn.heatmap(cm_df,
               annot=True,
               fmt='.2g',
               cbar=False,
               square=False,
               cmap="YlGnBu")
    plt.xlabel('\nPredicted')
    plt.ylabel('True\n')
    plt.title('LMU\n', fontweight='bold', fontsize=16)
    plt.yticks(rotation=0)
    plt.tight_layout()
    if save:
        plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
    plt.show()

sim.close()

<a id='Section_5'></a>
### Spiking LMU

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
net_type = 'slmu'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
model_name = "sLMU_{}_subset{}_{}".format(device,subset_window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['minibatch']
###########################################

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

y_train = y_train[:, None, None]
y_test = y_test[:, None, None]
y_val = y_val[:, None, None]

In [ ]:
with nengo.Network(seed=seed) as net:
    # remove some unnecessary features to speed up the training
    nengo_dl.configure_settings(
        trainable=None,
        stateful=False,
        keep_history=False,
    )

    # input node
    inp = nengo.Node(np.zeros(input_dim))
    
    order = int(network_parameters['order'])
    theta = network_parameters['theta']
    input_d = input_dim
    tau = network_parameters['tau'] 
    
    Q = np.arange(order, dtype=np.float64)
    R = (2 * Q + 1)[:, None] / theta
    j, i = np.meshgrid(Q, Q)
    A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R 
    B = (-1.0) ** Q[:, None] * R 
    C = np.ones((1, order))
    D = np.zeros((1,))

    disc_step = 1/theta
    A, B, _, _, _ = cont2discrete((A, B, C, D), dt=disc_step, method="zoh")
    
    A_H = 1/(1-np.exp(-disc_step/tau)) * (A - np.exp(-disc_step/tau)*np.identity(order))
    B_H = 1/(1-np.exp(-disc_step/tau)) * B

    for conn in net.all_connections:
        conn.synapse = network_parameters['synapse_all']

    max_rate = network_parameters['max_rate']
    amplitude = 1/max_rate
    lmu_inner = nengo.networks.EnsembleArray(n_neurons=int(network_parameters['n_neurons']),
                                             n_ensembles=order, 
                                             neuron_type=nengo.SpikingRectifiedLinear(amplitude=amplitude),
                                             max_rates=nengo.dists.Choice([max_rate]))
    conn_inner = nengo.Connection(lmu_inner.output, lmu_inner.input, transform=A_H, synapse=tau)
    net.config[conn_inner].trainable = True
    
    conn_in = nengo.Connection(inp, lmu_inner.input, transform=np.ones((1, input_d))*B_H, synapse=network_parameters['synapse_in'])
    net.config[conn_in].trainable = True
    
    # dense linear readout
    out = nengo.Node(size_in=n_classes)
    conn_out = nengo.Connection(lmu_inner.output, out, transform=nengo_dl.dists.Glorot(), synapse=network_parameters['synapse_out'])
    net.config[conn_out].trainable = True

    # record output
    p = nengo.Probe(out)

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=minibatch_train) as sim: 
    
    lmuEns_model_summary = sim.keras_model
    lmuEns_params = sum(np.prod(s.shape) for s in lmuEns_model_summary.weights)
    lmuEns_trainable_params = sum(np.prod(w.shape) for w in lmuEns_model_summary.trainable_weights)
    mem_fp, total, missed = memory_footprint(sim) 
    print('\n=================================================================')
    print('Total params:','{:,d}'.format(lmuEns_params))
    print('Trainable params:','{:,d}'.format(lmuEns_trainable_params))
    print('Non-trainable params:','{:,d}'.format(lmuEns_params-lmuEns_trainable_params))
    print('_________________________________________________________________\n')
    
    mem_fp, total, missed = memory_footprint(sim)

    print('Memory footprint (MB):',np.round(mem_fp,4))
    print('Total:',total)
    print('Missed:',missed)
    print("\n")
    
    sim.compile(
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam(network_parameters['lr']),
                metrics=["accuracy"],
               )
    
    sim.load_params("./output/tmp_s{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
    
    test = sim.evaluate(x_test, y_test)["probe_accuracy"]
    print("Test accuracy: "+str(np.round(test*100,2))+"%")
    
    prediction = sim.predict(x_test)
        
    predictions = list(prediction.values())[0]
    pred = predictions.argmax(axis=-1)
    
    save = False
    
    cm = confusion_matrix(y_test[:np.min([len(y_test), len(pred)]),-1,-1], pred[:np.min([len(y_test), len(pred)]),-1], normalize='true')
    labels = ConfusionMatrix_labels(subset)
    cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
    plt.figure(figsize=(7,5.25))
    sn.heatmap(cm_df,
               annot=True,
               fmt='.2g',
               cbar=False,
               square=False,
               cmap="YlGnBu")
    plt.xlabel('\nPredicted')
    plt.ylabel('True\n')
    plt.title('Spiking LMU\n', fontweight='bold', fontsize=16)
    plt.yticks(rotation=0)
    plt.tight_layout()
    if save:
        plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
    plt.show()

sim.close()

In [ ]:
get_sops_LMUens(net)

<a id='Section_6'></a>
### LMU (ff)

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
net_type = 'LMU'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
model_name = "LMU_freqdec_{}_subset{}_{}".format(device,subset_window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['minibatch']
###########################################

In [ ]:
freq_dec = True

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)

if freq_dec:
    x_train = frequency_decomposition(x_train)
    x_val = frequency_decomposition(x_val)
    x_test = frequency_decomposition(x_test)

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

y_train = y_train[:, None, None]
y_test = y_test[:, None, None]
y_val = y_val[:, None, None]

In [ ]:
with nengo.Network(seed=seed) as net:
    # remove some unnecessary features to speed up the training
    nengo_dl.configure_settings(
                                trainable=None,
                                stateful=False,
                                keep_history=False,
                               )

    # input node
    inp = nengo.Node(np.zeros(input_dim))

    # lmu cell
    lmu = LMUCell(
                  units=int(network_parameters['units']),
                  order=int(network_parameters['order']), 
                  theta=network_parameters['theta'], 
                  input_d=input_dim,
                  tau=network_parameters['tau'],
                )
    conn_in = nengo.Connection(inp, lmu.x, synapse=network_parameters['synapse_in'])
    net.config[conn_in].trainable = True

    # dense linear readout
    out = nengo.Node(size_in=n_classes)
    conn_out = nengo.Connection(lmu.h, out, transform=nengo_dl.dists.Glorot(), synapse=network_parameters['synapse_out'])
    net.config[conn_out].trainable = True

    # record output
    p = nengo.Probe(out)

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=minibatch_train) as sim:
    
    lmu_model_summary = sim.keras_model
    lmu_params = sum(np.prod(s.shape) for s in lmu_model_summary.weights)
    lmu_trainable_params = sum(np.prod(w.shape) for w in lmu_model_summary.trainable_weights)
    mem_fp, total, missed = memory_footprint(sim) 
    print('\n=================================================================')
    print('Total params:','{:,d}'.format(lmu_params))
    print('Trainable params:','{:,d}'.format(lmu_trainable_params))
    print('Non-trainable params:','{:,d}'.format(lmu_params-lmu_trainable_params)) 
    print('_________________________________________________________________\n')
    
    mem_fp, total, missed = memory_footprint(sim)

    print('Memory footprint (MB):',np.round(mem_fp,4))
    print('Total:',total)
    print('Missed:',missed)
    print("\n")
    
    sim.compile(
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam(network_parameters['lr']),
                metrics=["accuracy"],
               )
    
    sim.load_params("./output/tmp_{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
    
    test = sim.evaluate(x_test, y_test)["probe_accuracy"]
    print("Test accuracy: "+str(np.round(test*100,2))+"%")
    
    prediction = sim.predict(x_test)
        
    predictions = list(prediction.values())[0]
    pred = predictions.argmax(axis=-1)
    
    save = True
    
    cm = confusion_matrix(y_test[:np.min([len(y_test), len(pred)]),-1,-1], pred[:np.min([len(y_test), len(pred)]),-1], normalize='true')
    labels = ConfusionMatrix_labels(subset)
    cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
    plt.figure(figsize=(7,5.25))
    sn.heatmap(cm_df,
               annot=True,
               fmt='.2g',
               cbar=False,
               square=False,
               cmap="YlGnBu")
    plt.xlabel('\nPredicted')
    plt.ylabel('True\n')
    plt.title('LMU (frequency filtering)\n', fontweight='bold', fontsize=16)
    plt.yticks(rotation=0)
    plt.tight_layout()
    if save:
        plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
    plt.show()

sim.close()

<a id='Section_7'></a>
### Spiking LMU (ff)

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
net_type = 'slmu'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
model_name = "sLMU_freqdec_{}_subset{}_{}".format(device,subset_window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['minibatch']
###########################################

In [ ]:
freq_dec = True

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)

if freq_dec:
    x_train = frequency_decomposition(x_train)
    x_val = frequency_decomposition(x_val)
    x_test = frequency_decomposition(x_test)

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

y_train = y_train[:, None, None]
y_test = y_test[:, None, None]
y_val = y_val[:, None, None]

In [ ]:
with nengo.Network(seed=seed) as net:
    # remove some unnecessary features to speed up the training
    nengo_dl.configure_settings(
        trainable=None,
        stateful=False,
        keep_history=False,
    )

    # input node
    inp = nengo.Node(np.zeros(input_dim))
    
    order = int(network_parameters['order'])
    theta = network_parameters['theta']
    input_d = input_dim
    tau = network_parameters['tau'] 
    
    Q = np.arange(order, dtype=np.float64)
    R = (2 * Q + 1)[:, None] / theta
    j, i = np.meshgrid(Q, Q)
    A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R 
    B = (-1.0) ** Q[:, None] * R 
    C = np.ones((1, order))
    D = np.zeros((1,))

    disc_step = 1/theta
    A, B, _, _, _ = cont2discrete((A, B, C, D), dt=disc_step, method="zoh") 
    
    A_H = 1/(1-np.exp(-disc_step/tau)) * (A - np.exp(-disc_step/tau)*np.identity(order))
    B_H = 1/(1-np.exp(-disc_step/tau)) * B

    for conn in net.all_connections:
        conn.synapse = network_parameters['synapse_all']

    max_rate = network_parameters['max_rate']
    amplitude = 1/max_rate
    lmu_inner = nengo.networks.EnsembleArray(n_neurons=int(network_parameters['n_neurons']),
                                             n_ensembles=order, 
                                             neuron_type=nengo.SpikingRectifiedLinear(amplitude=amplitude),
                                             max_rates=nengo.dists.Choice([max_rate]))
    conn_inner = nengo.Connection(lmu_inner.output, lmu_inner.input, transform=A_H, synapse=tau)
    net.config[conn_inner].trainable = True
    
    conn_in = nengo.Connection(inp, lmu_inner.input, transform=np.ones((1, input_d))*B_H, synapse=network_parameters['synapse_in'])
    net.config[conn_in].trainable = True
    
    # dense linear readout
    out = nengo.Node(size_in=n_classes)
    conn_out = nengo.Connection(lmu_inner.output, out, transform=nengo_dl.dists.Glorot(), synapse=network_parameters['synapse_out'])
    net.config[conn_out].trainable = True

    # record output
    p = nengo.Probe(out)

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=minibatch_train) as sim: 
    
    lmuEns_model_summary = sim.keras_model
    lmuEns_params = sum(np.prod(s.shape) for s in lmuEns_model_summary.weights)
    lmuEns_trainable_params = sum(np.prod(w.shape) for w in lmuEns_model_summary.trainable_weights)
    mem_fp, total, missed = memory_footprint(sim) 
    print('\n=================================================================')
    print('Total params:','{:,d}'.format(lmuEns_params))
    print('Trainable params:','{:,d}'.format(lmuEns_trainable_params))
    print('Non-trainable params:','{:,d}'.format(lmuEns_params-lmuEns_trainable_params)) 
    print('_________________________________________________________________\n')
    
    mem_fp, total, missed = memory_footprint(sim)

    print('Memory footprint (MB):',np.round(mem_fp,4))
    print('Total:',total)
    print('Missed:',missed)
    print("\n")
    
    sim.compile(
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam(network_parameters['lr']),
                metrics=["accuracy"],
               )
    
    sim.load_params("./output/tmp_s{}_{}_{}/best_test_{}".format(net_type,snn_nni_experiment,datafile[5:],snn_nni_experiment))
    
    test = sim.evaluate(x_test, y_test)["probe_accuracy"]
    print("Test accuracy: "+str(np.round(test*100,2))+"%")
    
    prediction = sim.predict(x_test)
        
    predictions = list(prediction.values())[0]
    pred = predictions.argmax(axis=-1)
    
    save = True
    
    cm = confusion_matrix(y_test[:np.min([len(y_test), len(pred)]),-1,-1], pred[:np.min([len(y_test), len(pred)]),-1], normalize='true')
    labels = ConfusionMatrix_labels(subset)
    cm_df = pd.DataFrame(cm, index=[ii for ii in labels], columns=[jj for jj in labels])
    plt.figure(figsize=(7,5.25))
    sn.heatmap(cm_df,
               annot=True,
               fmt='.2g',
               cbar=False,
               square=False,
               cmap="YlGnBu")
    plt.xlabel('\nPredicted')
    plt.ylabel('True\n')
    plt.title('Spiking LMU (frequency filtering)\n', fontweight='bold', fontsize=16)
    plt.yticks(rotation=0)
    plt.tight_layout()
    if save:
        plt.savefig('./pictures/'+model_name+'_'+optim_nni_experiment+'_'+optim_nni_trial+'.png')
    plt.show()

sim.close()

In [ ]:
get_sops_LMUens(net, freq_dec)

<a id='Section_8'></a>
### <i>FLOPs calculation and energy estimation for LMU and LMU (ff)</i>

In [ ]:
"""
Core classes for the KerasLMU package.
"""

import numpy as np
import tensorflow as tf
from packaging import version

if version.parse(tf.__version__) < version.parse("2.6.0rc0"):
    from tensorflow.python.keras.layers.recurrent import DropoutRNNCellMixin
else:
    from keras.layers.recurrent import DropoutRNNCellMixin


class LMUCell(DropoutRNNCellMixin, tf.keras.layers.Layer):
    """
    Implementation of LMU cell (to be used within Keras RNN wrapper).
    In general, the LMU cell consists of two parts: a memory component (decomposing
    the input signal using Legendre polynomials as a basis), and a hidden component
    (learning nonlinear mappings from the memory component). [1]_ [2]_
    This class processes one step within the whole time sequence input. Use the ``LMU``
    class to create a recurrent Keras layer to process the whole sequence. Calling
    ``LMU()`` is equivalent to doing ``RNN(LMUCell())``.
    Parameters
    ----------
    memory_d : int
        Dimensionality of input to memory component.
    order : int
        The number of degrees in the transfer function of the LTI system used to
        represent the sliding window of history. This parameter sets the number of
        Legendre polynomials used to orthogonally represent the sliding window.
    theta : float
        The number of timesteps in the sliding window that is represented using the
        LTI system. In this context, the sliding window represents a dynamic range of
        data, of fixed size, that will be used to predict the value at the next time
        step. If this value is smaller than the size of the input sequence, only that
        number of steps will be represented at the time of prediction, however the
        entire sequence will still be processed in order for information to be
        projected to and from the hidden layer. If ``trainable_theta`` is enabled, then
        theta will be updated during the course of training.
    hidden_cell : ``tf.keras.layers.Layer``
        Keras Layer/RNNCell implementing the hidden component.
    trainable_theta : bool
        If True, theta is learnt over the course of training. Otherwise, it is kept
        constant.
    hidden_to_memory : bool
        If True, connect the output of the hidden component back to the memory
        component (default False).
    memory_to_memory : bool
        If True, add a learnable recurrent connection (in addition to the static
        Legendre system) to the memory component (default False).
    input_to_hidden : bool
        If True, connect the input directly to the hidden component (in addition to
        the connection from the memory component) (default False).
    discretizer : str
        The method used to discretize the A and B matrices of the LMU. Current
        options are "zoh" (short for Zero Order Hold) and "euler".
        "zoh" is more accurate, but training will be slower than "euler" if
        ``trainable_theta=True``. Note that a larger theta is needed when discretizing
        using "euler" (a value that is larger than ``4*order`` is recommended).
    kernel_initializer : ``tf.initializers.Initializer``
        Initializer for weights from input to memory/hidden component. If ``None``,
        no weights will be used, and the input size must match the memory/hidden size.
    recurrent_initializer : ``tf.initializers.Initializer``
        Initializer for ``memory_to_memory`` weights (if that connection is enabled).
    dropout : float
        Dropout rate on input connections.
    recurrent_dropout : float
        Dropout rate on ``memory_to_memory`` connection.
    References
    ----------
    .. [1] Voelker and Eliasmith (2018). Improving spiking dynamical
       networks: Accurate delays, higher-order synapses, and time cells.
       Neural Computation, 30(3): 569-609.
    .. [2] Voelker and Eliasmith. "Methods and systems for implementing
       dynamic neural networks." U.S. Patent Application No. 15/243,223.
       Filing date: 2016-08-22.
    """

    def __init__(
        self,
        memory_d,
        order,
        theta,
        hidden_cell,
        trainable_theta=False,
        hidden_to_memory=False,
        memory_to_memory=False,
        input_to_hidden=False,
        discretizer="zoh",
        kernel_initializer="glorot_uniform",
        recurrent_initializer="orthogonal",
        dropout=0,
        recurrent_dropout=0,
        tau=0.001, 
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.memory_d = memory_d
        self.order = order
        self._init_theta = theta
        self.hidden_cell = hidden_cell
        self.trainable_theta = trainable_theta
        self.hidden_to_memory = hidden_to_memory
        self.memory_to_memory = memory_to_memory
        self.input_to_hidden = input_to_hidden
        self.discretizer = discretizer
        self.kernel_initializer = kernel_initializer
        self.recurrent_initializer = recurrent_initializer
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.tau = tau

        self.kernel = None
        self.recurrent_kernel = None
        self.theta_inv = None
        self.A = None
        self.B = None

        if self.discretizer not in ("zoh", "euler"):
            raise ValueError(
                f"discretizer must be 'zoh' or 'euler' (got '{self.discretizer}')"
            )

        if self.hidden_cell is None:
            for conn in ("hidden_to_memory", "input_to_hidden"):
                if getattr(self, conn):
                    raise ValueError(f"{conn} must be False if hidden_cell is None")

            self.hidden_output_size = self.memory_d * self.order
            self.hidden_state_size = []
        elif hasattr(self.hidden_cell, "state_size"):
            self.hidden_output_size = self.hidden_cell.output_size
            self.hidden_state_size = self.hidden_cell.state_size
        else:
            # TODO: support layers that don't have the `units` attribute
            self.hidden_output_size = self.hidden_cell.units
            self.hidden_state_size = [self.hidden_cell.units]

        self.state_size = tf.nest.flatten(self.hidden_state_size) + [
            self.memory_d * self.order
        ]
        self.output_size = self.hidden_output_size

    @property
    def theta(self):
        """
        Value of the ``theta`` parameter.
        If ``trainable_theta=True`` this returns the trained value, not the initial
        value passed in to the constructor.
        """
        if self.built:
            return 1 / tf.keras.backend.get_value(self.theta_inv)

        return self._init_theta

    def _gen_AB(self):
        """Generates A and B matrices."""

        # compute analog A/B matrices
        Q = np.arange(self.order, dtype=np.float64)
        R = (2 * Q + 1)[:, None] / self._init_theta
        j, i = np.meshgrid(Q, Q)
        A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
        B = (-1.0) ** Q[:, None] * R
        
        disc_step = 1/self._init_theta
        A = 1/(1-np.exp(-disc_step/self.tau)) * (A - np.exp(-disc_step/self.tau)*np.identity(self.order))  
        B = 1/(1-np.exp(-disc_step/self.tau)) * B  

        # discretize matrices
        if self.discretizer == "zoh":
            # save the un-discretized matrices for use in .call
            self._base_A = tf.constant(A.T, dtype=self.dtype)
            self._base_B = tf.constant(B.T, dtype=self.dtype)
            
            disc_step = 1/self._init_theta
            self.A, self.B = LMUCell._cont2discrete_zoh(
                self._base_A / self._init_theta, self._base_B / self._init_theta, disc_step
            )
        else:
            if not self.trainable_theta:
                A = A / self._init_theta + np.eye(self.order)
                B = B / self._init_theta

            self.A = tf.constant(A.T, dtype=self.dtype)
            self.B = tf.constant(B.T, dtype=self.dtype)

    @staticmethod
    def _cont2discrete_zoh(A, B, dt):
        """
        Function to discretize A and B matrices using Zero Order Hold method.
        Functionally equivalent to
        ``scipy.signal.cont2discrete((A.T, B.T, _, _), method="zoh", dt=1.0)``
        (but implemented in TensorFlow so that it is differentiable).
        Note that this accepts and returns matrices that are transposed from the
        standard linear system implementation (as that makes it easier to use in
        `.call`).
        """

        # combine A/B and pad to make square matrix
        em_upper = tf.concat([A, B], axis=0)
        em = tf.pad(em_upper, [(0, 0), (0, B.shape[0])])

        # compute matrix exponential
        ms = tf.linalg.expm(dt*em)

        # slice A/B back out of combined matrix
        discrt_A = ms[: A.shape[0], : A.shape[1]]
        discrt_B = ms[A.shape[0] :, : A.shape[1]]

        return discrt_A, discrt_B

    def build(self, input_shape):
        """
        Builds the cell.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        super().build(input_shape)

        enc_d = input_shape[-1]
        if self.hidden_to_memory:
            enc_d += self.hidden_output_size

        if self.kernel_initializer is not None:
            self.kernel = self.add_weight(
                name="kernel",
                shape=(enc_d, self.memory_d),
                initializer=self.kernel_initializer,
            )
        else:
            self.kernel = None
            if enc_d != self.memory_d:
                raise ValueError(
                    f"For LMUCells with no input kernel, the input dimension ({enc_d})"
                    f" must equal `memory_d` ({self.memory_d})."
                )

        # when using euler, 1/theta results in better gradients for the memory
        # update since you are multiplying 1/theta, as compared to dividing theta
        if self.trainable_theta:
            self.theta_inv = self.add_weight(
                name="theta_inv",
                shape=(),
                initializer=tf.initializers.constant(1 / self._init_theta),
                constraint=tf.keras.constraints.NonNeg(),
            )
        else:
            self.theta_inv = tf.constant(1 / self._init_theta, dtype=self.dtype)

        if self.memory_to_memory:
            self.recurrent_kernel = self.add_weight(
                name="recurrent_kernel",
                shape=(self.memory_d * self.order, self.memory_d),
                initializer=self.recurrent_initializer,
            )
        else:
            self.recurrent_kernel = None

        if self.hidden_cell is not None and not self.hidden_cell.built:
            hidden_input_d = self.memory_d * self.order
            if self.input_to_hidden:
                hidden_input_d += input_shape[-1]
            with tf.name_scope(self.hidden_cell.name):
                self.hidden_cell.build((input_shape[0], hidden_input_d))

        # generate A and B matrices
        self._gen_AB()

    def call(self, inputs, states, training=None):
        """
        Apply this cell to inputs.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        if training is None:
            training = tf.keras.backend.learning_phase()

        states = tf.nest.flatten(states)

        # state for the hidden cell
        h = states[:-1]
        # state for the LMU memory
        m = states[-1]

        # compute memory input
        u_in = tf.concat((inputs, h[0]), axis=1) if self.hidden_to_memory else inputs
        if self.dropout > 0:
            u_in *= self.get_dropout_mask_for_cell(u_in, training)
        u = u_in if self.kernel is None else tf.matmul(u_in, self.kernel)

        if self.memory_to_memory:
            if self.recurrent_dropout > 0:
                # note: we don't apply dropout to the memory input, only
                # the recurrent kernel
                rec_m = m * self.get_recurrent_dropout_mask_for_cell(m, training)
            else:
                rec_m = m

            u += tf.matmul(rec_m, self.recurrent_kernel)

        # separate memory/order dimensions
        m = tf.reshape(m, (-1, self.memory_d, self.order))
        u = tf.expand_dims(u, -1)

        # update memory
        if self.discretizer == "zoh" and self.trainable_theta:
            # apply updated theta and re-discretize
            A, B = LMUCell._cont2discrete_zoh(
                self._base_A * self.theta_inv, self._base_B * self.theta_inv
            )
        else:
            A, B = self.A, self.B

        _m = tf.matmul(m, A) + tf.matmul(u, B)

        if self.discretizer == "euler" and self.trainable_theta:
            # apply updated theta. this is the same as scaling A/B by theta, but it's
            # more efficient to do it this way.
            # note that when computing this way the A matrix does not
            # include the identity matrix along the diagonal (since we don't want to
            # scale that part by theta), which is why we do += instead of =
            m += _m * self.theta_inv
        else:
            m = _m

        # re-combine memory/order dimensions
        m = tf.reshape(m, (-1, self.memory_d * self.order))

        # apply hidden cell
        h_in = tf.concat((m, inputs), axis=1) if self.input_to_hidden else m

        if self.hidden_cell is None:
            o = h_in
            h = []
        elif hasattr(self.hidden_cell, "state_size"):
            o, h = self.hidden_cell(h_in, h, training=training)
        else:
            o = self.hidden_cell(h_in, training=training)
            h = [o]

        return o, h + [m]

    def reset_dropout_mask(self):
        """Reset dropout mask for memory and hidden components."""
        super().reset_dropout_mask()
        if isinstance(self.hidden_cell, DropoutRNNCellMixin):
            self.hidden_cell.reset_dropout_mask()

    def reset_recurrent_dropout_mask(self):
        """Reset recurrent dropout mask for memory and hidden components."""
        super().reset_recurrent_dropout_mask()
        if isinstance(self.hidden_cell, DropoutRNNCellMixin):
            self.hidden_cell.reset_recurrent_dropout_mask()

    def get_config(self):
        """Return config of layer (for serialization during model saving/loading)."""

        config = super().get_config()
        config.update(
            dict(
                memory_d=self.memory_d,
                order=self.order,
                theta=self._init_theta,
                hidden_cell=tf.keras.layers.serialize(self.hidden_cell),
                trainable_theta=self.trainable_theta,
                hidden_to_memory=self.hidden_to_memory,
                memory_to_memory=self.memory_to_memory,
                input_to_hidden=self.input_to_hidden,
                discretizer=self.discretizer,
                kernel_initializer=self.kernel_initializer,
                recurrent_initializer=self.recurrent_initializer,
                dropout=self.dropout,
                recurrent_dropout=self.recurrent_dropout,
            )
        )

        return config

    @classmethod
    def from_config(cls, config):
        """Load model from serialized config."""

        config["hidden_cell"] = tf.keras.layers.deserialize(config["hidden_cell"])
        return super().from_config(config)


class LMU(tf.keras.layers.Layer):
    """
    A layer of trainable low-dimensional delay systems.
    Each unit buffers its encoded input
    by internally representing a low-dimensional
    (i.e., compressed) version of the sliding window.
    Nonlinear decodings of this representation,
    expressed by the A and B matrices, provide
    computations across the window, such as its
    derivative, energy, median value, etc ([1]_, [2]_).
    Note that these decoder matrices can span across
    all of the units of an input sequence.
    Parameters
    ----------
    memory_d : int
        Dimensionality of input to memory component.
    order : int
        The number of degrees in the transfer function of the LTI system used to
        represent the sliding window of history. This parameter sets the number of
        Legendre polynomials used to orthogonally represent the sliding window.
    theta : float
        The number of timesteps in the sliding window that is represented using the
        LTI system. In this context, the sliding window represents a dynamic range of
        data, of fixed size, that will be used to predict the value at the next time
        step. If this value is smaller than the size of the input sequence, only that
        number of steps will be represented at the time of prediction, however the
        entire sequence will still be processed in order for information to be
        projected to and from the hidden layer. If ``trainable_theta`` is enabled, then
        theta will be updated during the course of training.
    hidden_cell : ``tf.keras.layers.Layer``
        Keras Layer/RNNCell implementing the hidden component.
    trainable_theta : bool
        If True, theta is learnt over the course of training. Otherwise, it is kept
        constant.
    hidden_to_memory : bool
        If True, connect the output of the hidden component back to the memory
        component (default False).
    memory_to_memory : bool
        If True, add a learnable recurrent connection (in addition to the static
        Legendre system) to the memory component (default False).
    input_to_hidden : bool
        If True, connect the input directly to the hidden component (in addition to
        the connection from the memory component) (default False).
    discretizer : str
        The method used to discretize the A and B matrices of the LMU. Current
        options are "zoh" (short for Zero Order Hold) and "euler".
        "zoh" is more accurate, but training will be slower than "euler" if
        ``trainable_theta=True``. Note that a larger theta is needed when discretizing
        using "euler" (a value that is larger than ``4*order`` is recommended).
    kernel_initializer : ``tf.initializers.Initializer``
        Initializer for weights from input to memory/hidden component. If ``None``,
        no weights will be used, and the input size must match the memory/hidden size.
    recurrent_initializer : ``tf.initializers.Initializer``
        Initializer for ``memory_to_memory`` weights (if that connection is enabled).
    dropout : float
        Dropout rate on input connections.
    recurrent_dropout : float
        Dropout rate on ``memory_to_memory`` connection.
    return_sequences : bool, optional
        If True, return the full output sequence. Otherwise, return just the last
        output in the output sequence.
    References
    ----------
    .. [1] Voelker and Eliasmith (2018). Improving spiking dynamical
       networks: Accurate delays, higher-order synapses, and time cells.
       Neural Computation, 30(3): 569-609.
    .. [2] Voelker and Eliasmith. "Methods and systems for implementing
       dynamic neural networks." U.S. Patent Application No. 15/243,223.
       Filing date: 2016-08-22.
    """

    def __init__(
        self,
        memory_d,
        order,
        theta,
        hidden_cell,
        trainable_theta=False,
        hidden_to_memory=False,
        memory_to_memory=False,
        input_to_hidden=False,
        discretizer="zoh",
        kernel_initializer="glorot_uniform",
        recurrent_initializer="orthogonal",
        dropout=0,
        recurrent_dropout=0,
        return_sequences=False,
        tau=0.001,
        **kwargs,
    ):

        super().__init__(**kwargs)

        self.memory_d = memory_d
        self.order = order
        self._init_theta = theta
        self.hidden_cell = hidden_cell
        self.trainable_theta = trainable_theta
        self.hidden_to_memory = hidden_to_memory
        self.memory_to_memory = memory_to_memory
        self.input_to_hidden = input_to_hidden
        self.discretizer = discretizer
        self.kernel_initializer = kernel_initializer
        self.recurrent_initializer = recurrent_initializer
        self.dropout = dropout
        self.recurrent_dropout = recurrent_dropout
        self.return_sequences = return_sequences
        self.layer = None
        self.tau = tau

    @property
    def theta(self):
        """
        Value of the ``theta`` parameter.
        If ``trainable_theta=True`` this returns the trained value, not the initial
        value passed in to the constructor.
        """

        if self.built:
            return (
                self.layer.theta
                if isinstance(self.layer, LMUFeedforward)
                else self.layer.cell.theta
            )

        return self._init_theta

    def build(self, input_shapes):
        """
        Builds the layer.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        super().build(input_shapes)

        if (
            not self.hidden_to_memory
            and not self.memory_to_memory
            and input_shapes[1] is not None
            and not self.trainable_theta
        ):
            self.layer = LMUFeedforward(
                memory_d=self.memory_d,
                order=self.order,
                theta=self._init_theta,
                hidden_cell=self.hidden_cell,
                input_to_hidden=self.input_to_hidden,
                discretizer=self.discretizer,
                kernel_initializer=self.kernel_initializer,
                dropout=self.dropout,
                return_sequences=self.return_sequences,
            )
        else:
            self.layer = tf.keras.layers.RNN(
                LMUCell(
                    memory_d=self.memory_d,
                    order=self.order,
                    theta=self._init_theta,
                    hidden_cell=self.hidden_cell,
                    trainable_theta=self.trainable_theta,
                    hidden_to_memory=self.hidden_to_memory,
                    memory_to_memory=self.memory_to_memory,
                    input_to_hidden=self.input_to_hidden,
                    discretizer=self.discretizer,
                    kernel_initializer=self.kernel_initializer,
                    recurrent_initializer=self.recurrent_initializer,
                    dropout=self.dropout,
                    recurrent_dropout=self.recurrent_dropout,
                    tau=self.tau,
                ),
                return_sequences=self.return_sequences,
            )

        self.layer.build(input_shapes)

    def call(self, inputs, training=None):
        """
        Apply this layer to inputs.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        return self.layer.call(inputs, training=training)

    def get_config(self):
        """Return config of layer (for serialization during model saving/loading)."""

        config = super().get_config()
        config.update(
            dict(
                memory_d=self.memory_d,
                order=self.order,
                theta=self._init_theta,
                hidden_cell=tf.keras.layers.serialize(self.hidden_cell),
                trainable_theta=self.trainable_theta,
                hidden_to_memory=self.hidden_to_memory,
                memory_to_memory=self.memory_to_memory,
                input_to_hidden=self.input_to_hidden,
                discretizer=self.discretizer,
                kernel_initializer=self.kernel_initializer,
                recurrent_initializer=self.recurrent_initializer,
                dropout=self.dropout,
                recurrent_dropout=self.recurrent_dropout,
                return_sequences=self.return_sequences,
            )
        )

        return config

    @classmethod
    def from_config(cls, config):
        """Load model from serialized config."""

        config["hidden_cell"] = tf.keras.layers.deserialize(config["hidden_cell"])
        return super().from_config(config)


class LMUFeedforward(tf.keras.layers.Layer):
    """
    Layer class for the feedforward variant of the LMU.
    This class assumes no recurrent connections are desired in the memory component.
    Produces the output of the delay system by evaluating the convolution of the input
    sequence with the impulse response from the LMU cell.
    Parameters
    ----------
    memory_d : int
        Dimensionality of input to memory component.
    order : int
        The number of degrees in the transfer function of the LTI system used to
        represent the sliding window of history. This parameter sets the number of
        Legendre polynomials used to orthogonally represent the sliding window.
    theta : float
        The number of timesteps in the sliding window that is represented using the
        LTI system. In this context, the sliding window represents a dynamic range of
        data, of fixed size, that will be used to predict the value at the next time
        step. If this value is smaller than the size of the input sequence, only that
        number of steps will be represented at the time of prediction, however the
        entire sequence will still be processed in order for information to be
        projected to and from the hidden layer.
    hidden_cell : ``tf.keras.layers.Layer``
        Keras Layer implementing the hidden component.
    input_to_hidden : bool
        If True, connect the input directly to the hidden component (in addition to
        the connection from the memory component) (default False).
    discretizer : str
        The method used to discretize the A and B matrices of the LMU. Current
        options are "zoh" (short for Zero Order Hold) and "euler".
        "zoh" is more accurate, but training will be slower than "euler" if
        ``trainable_theta=True``. Note that a larger theta is needed when discretizing
        using "euler" (a value that is larger than ``4*order`` is recommended).
    kernel_initializer : ``tf.initializers.Initializer``
        Initializer for weights from input to memory/hidden component. If ``None``,
        no weights will be used, and the input size must match the memory/hidden size.
    dropout : float
        Dropout rate on input connections.
    return_sequences : bool, optional
        If True, return the full output sequence. Otherwise, return just the last
        output in the output sequence.
    conv_mode : "fft" or "raw"
        The method for performing the inpulse response convolution. "fft" uses FFT
        convolution (default). "raw" uses explicit convolution, which may be faster
        for particular models on particular hardware.
    truncate_ir : float
        The portion of the impulse response to truncate when using "raw"
        convolution (see ``conv_mode``). This is an approximate upper bound on the error
        relative to the exact implementation. Smaller ``theta`` values result in more
        truncated elements for a given value of ``truncate_ir``, improving efficiency.
    """

    def __init__(
        self,
        memory_d,
        order,
        theta,
        hidden_cell,
        input_to_hidden=False,
        discretizer="zoh",
        kernel_initializer="glorot_uniform",
        dropout=0,
        return_sequences=False,
        conv_mode="fft",
        truncate_ir=1e-4,
        **kwargs,
    ):
        super().__init__(**kwargs)

        if input_to_hidden and hidden_cell is None:
            raise ValueError("input_to_hidden must be False if hidden_cell is None")

        if conv_mode not in ("fft", "raw"):
            raise ValueError(f"Unrecognized conv mode '{conv_mode}'")

        self.memory_d = memory_d
        self.order = order
        self.theta = theta
        self.hidden_cell = hidden_cell
        self.input_to_hidden = input_to_hidden
        self.discretizer = discretizer
        self.kernel_initializer = kernel_initializer
        self.dropout = dropout
        self.return_sequences = return_sequences
        self.conv_mode = conv_mode.lower()
        self.truncate_ir = truncate_ir

        # create a standard LMUCell to generate the impulse response during `build`
        self.delay_layer = tf.keras.layers.RNN(
            LMUCell(
                memory_d=1,
                order=order,
                theta=theta,
                hidden_cell=None,
                trainable_theta=False,
                input_to_hidden=False,
                hidden_to_memory=False,
                memory_to_memory=False,
                discretizer=discretizer,
                kernel_initializer=None,
                trainable=False,
            ),
            return_sequences=True,
        )

    def build(self, input_shape):
        """
        Builds the layer.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        super().build(input_shape)

        seq_len = input_shape[1]
        enc_d = input_shape[-1]

        if seq_len is None:
            # TODO: we could dynamically run the impulse response for longer if
            #  needed using stateful=True
            raise ValueError(
                f"LMUFeedforward requires that the input shape's temporal axis be "
                f"fully specified (got {seq_len})"
            )

        impulse = tf.reshape(tf.eye(seq_len, 1), (1, -1, 1))

        self.impulse_response = tf.squeeze(
            self.delay_layer(impulse, training=False), axis=0
        )

        if self.conv_mode == "fft":
            self.impulse_response = tf.signal.rfft(
                tf.transpose(self.impulse_response),
                fft_length=[2 * seq_len],
            )
        else:
            if self.truncate_ir is not None:
                assert self.impulse_response.shape == (seq_len, self.order)

                cumsum = tf.math.cumsum(
                    tf.math.abs(self.impulse_response), axis=0, reverse=True
                )
                cumsum = cumsum / cumsum[0]
                to_drop = tf.reduce_all(cumsum < self.truncate_ir, axis=-1)
                if to_drop[-1]:
                    cutoff = tf.where(to_drop)[0, -1]
                    self.impulse_response = self.impulse_response[:cutoff]

            self.impulse_response = tf.reshape(
                self.impulse_response,
                (self.impulse_response.shape[0], 1, 1, self.order),
            )
            self.impulse_response = self.impulse_response[::-1, :, :, :]

        if self.kernel_initializer is not None:
            self.kernel = self.add_weight(
                name="kernel",
                shape=(input_shape[-1], self.memory_d),
                initializer=self.kernel_initializer,
            )
        else:
            self.kernel = None
            if enc_d != self.memory_d:
                raise ValueError(
                    f"For LMUCells with no input kernel, the input dimension ({enc_d})"
                    f" must equal `memory_d` ({self.memory_d})."
                )

        if self.hidden_cell is not None and not self.hidden_cell.built:
            hidden_input_d = self.memory_d * self.order
            if self.input_to_hidden:
                hidden_input_d += input_shape[-1]
            with tf.name_scope(self.hidden_cell.name):
                self.hidden_cell.build((input_shape[0], hidden_input_d))

    def call(self, inputs, training=None):
        """
        Apply this layer to inputs.
        Notes
        -----
        This method should not be called manually; rather, use the implicit layer
        callable behaviour (like ``my_layer(inputs)``), which will apply this method
        with some additional bookkeeping.
        """

        if training is None:
            training = tf.keras.backend.learning_phase()

        if self.dropout:
            inputs = tf.keras.layers.Dropout(
                self.dropout, noise_shape=(inputs.shape[0], 1) + inputs.shape[2:]
            )(inputs)

        # Apply input encoders
        u = (
            inputs
            if self.kernel is None
            else tf.matmul(inputs, self.kernel, name="input_encoder_mult")
        )

        if self.conv_mode == "fft":
            m = self._fft_convolution(u)
        elif self.conv_mode == "raw":
            m = self._raw_convolution(u)

        # apply hidden cell
        h_in = tf.concat((m, inputs), axis=-1) if self.input_to_hidden else m

        if self.hidden_cell is None:
            h = h_in if self.return_sequences else h_in[:, -1]
        elif hasattr(self.hidden_cell, "state_size"):
            h = tf.keras.layers.RNN(
                self.hidden_cell, return_sequences=self.return_sequences
            )(h_in, training=training)
        else:
            if not self.return_sequences:
                # no point applying the hidden cell to the whole sequence
                h = self.hidden_cell(h_in[:, -1], training=training)
            else:
                h = tf.keras.layers.TimeDistributed(self.hidden_cell)(
                    h_in, training=training
                )

        return h

    def _fft_convolution(self, u):
        seq_len = tf.shape(u)[1]

        # FFT requires shape (batch, memory_d, timesteps)
        u = tf.transpose(u, perm=[0, 2, 1])

        # Pad sequences to avoid circular convolution
        # Perform the FFT
        fft_input = tf.signal.rfft(u, fft_length=[2 * seq_len])

        # Elementwise product of FFT (with broadcasting)
        result = tf.expand_dims(fft_input, axis=-2) * self.impulse_response

        # Inverse FFT
        m = tf.signal.irfft(result, fft_length=[2 * seq_len])[..., :seq_len]

        m = tf.reshape(m, (-1, self.order * self.memory_d, seq_len))

        return tf.transpose(m, perm=[0, 2, 1])

    def _raw_convolution(self, u):
        seq_len = tf.shape(u)[1]
        ir_len = self.impulse_response.shape[0]

        u = tf.expand_dims(u, -1)
        m = tf.nn.conv2d(
            u,
            self.impulse_response,
            strides=1,
            data_format="NHWC",
            padding=[[0, 0], [ir_len - 1, 0], [0, 0], [0, 0]],
        )
        m = tf.reshape(m, (-1, seq_len, self.memory_d * self.order))
        return m

    def get_config(self):
        """Return config of layer (for serialization during model saving/loading)."""

        config = super().get_config()
        config.update(
            dict(
                memory_d=self.memory_d,
                order=self.order,
                theta=self.theta,
                hidden_cell=tf.keras.layers.serialize(self.hidden_cell),
                input_to_hidden=self.input_to_hidden,
                discretizer=self.discretizer,
                kernel_initializer=self.kernel_initializer,
                dropout=self.dropout,
                return_sequences=self.return_sequences,
                conv_mode=self.conv_mode,
                truncate_ir=self.truncate_ir,
            )
        )

        return config

    @classmethod
    def from_config(cls, config):
        """Load model from serialized config."""

        config["hidden_cell"] = tf.keras.layers.deserialize(config["hidden_cell"])
        return super().from_config(config)

In [ ]:
tf.keras.backend.clear_session()

# set seed to ensure the examples are reproducible
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
rng = np.random.RandomState(seed)

In [ ]:
freq_dec = False

In [ ]:
net_type = 'lmu'

device = 'watch'
subset = 2
time_window = 2

window_size = 20*time_window # 20 Hz sampling times the temporal length of the window

datafile = 'data_'+device+'_subset'+str(subset)+'_'+str(window_size)

In [ ]:
if freq_dec:
    model_name = "LMU_freqdec_{}_subset{}_{}".format(device,subset_window_size)
else:
    model_name = "LMU_{}_subset{}_{}".format(device,subset_window_size)

In [ ]:
##### GET NETWORK STRUCTURE PARAMETERS #####
optim_nni_experiment = ''
optim_nni_trial = ''
optim_filename = 'parameter.cfg'
optim_nni_ref = 'nni-experiments/'+optim_nni_experiment+'/trials/'+optim_nni_trial
optim_nni_dir = os.path.expanduser('~')
optim_filepath = os.path.join(optim_nni_dir,optim_nni_ref,optim_filename)

with open(optim_filepath, 'r') as f:
    data = f.read()

params = json.loads(data)
network_parameters = params['parameters']

minibatch_train = network_parameters['minibatch']
###########################################

In [ ]:
(x_train, x_val, x_test, y_train_oh, y_val_oh, y_test_oh) = load_wisdm2_data(datafile)

if freq_dec:
    from scipy.signal import butter
    x_train = frequency_decomposition(x_train)
    x_val = frequency_decomposition(x_val)
    x_test = frequency_decomposition(x_test)

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train_oh[0])

y_train = np.argmax(y_train_oh, axis=-1)
y_val = np.argmax(y_val_oh, axis=-1)
y_test = np.argmax(y_test_oh, axis=-1)

print('timesteps:',timesteps)
print('input_dim:',input_dim)
print('n_classes:',n_classes)

y_train = y_train[:, None, None]
y_test = y_test[:, None, None]
y_val = y_val[:, None, None]

In [ ]:
model = Sequential()

model.add(Input((timesteps,input_dim)))
model.add(LMU(memory_d=1,
              order=int(network_parameters['order']),
              theta=network_parameters['theta'],
              hidden_cell=tf.keras.layers.SimpleRNNCell(units=int(network_parameters['units'])),
              hidden_to_memory=False,
              memory_to_memory=True,
              input_to_hidden=True,
              tau=network_parameters['tau'],
             )
         )
model.add(Dense(n_classes, use_bias=False))

model.summary()

In [ ]:
print("FLOPs: {}".format(get_flops(model)))

In [ ]:
energy = get_flops(model)*7.53e-10 # Event-Driven Signal Processing with Neuromorphic Computing Systems, https://ieeexplore.ieee.org/document/9053043/

print("Energy evaluation on Movidius: "+str(np.round(energy*1e6,2))+" μJ")